<a href="https://colab.research.google.com/github/FatemehAbediK/signal-processing-EEG/blob/main/Teaching_signal_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##mne.tools/mne study template , can be a help!!

##Anaconda python----MNE environment---jupyter lab

## conda activate mne
## conda install jupyter lab

In [ ]:
import matplotlib
import pathlib
import mne
###Ensure Matplotlib uses the Qt5Agg backend,
## which is the best choice for MNE-Python's interactive plotting functions.
matplotlib.use('Qt5Agg')


In [ ]:
###Retrieve the storage location of the sample data, and download the dataset if it cannot be found.
sample_data_dir = mne.datasets.sample.data_path()
# Convert to a pathlib.Path for more convenience
sample_data_dir = pathlib.Path(sample_data_dir)
sample_data_dir


In [ ]:
##Load some raw data!
raw_path = sample_data_dir / '.....fif'
raw = mne.io.read_raw(raw_path)
raw

raw.plot()
#we go to the new window of the EEg so like the eeg lab it is great & it's interactive!!

In [ ]:
##Extract events from the channels
events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}
event_id

In [ ]:
##Plot the raw data again, but add event markers----it will addd the vertical lines with the names of event-ids
raw.plot(events=events, event_id=event_id)


In [ ]:
raw.info
raw.info['meas_date']
raw.info['sfreq']
raw.info['bads']
raw.ch_names


In [ ]:
#example:

raw.ch_names[:10]

raw.info['chs'][0]


In [ ]:
###Visualize the sensor locations
raw.plot_sensors(ch_type='eeg')
raw.plot_sensors(kind='3d', ch_type='eeg')

In [ ]:
###Mark channels as bad
#Mark an additional EEG channel as bad and view the topoplot.

raw.info['bads']
raw.info['bads'] += ['EEG 051']

raw.plot_sensors(ch_type='eeg')

In [ ]:
##Select only a subset of the channels(eeg & eog)----but first we make a copy then pick our certain types
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])      #exclude :do not delete anything !give us all even the bad channels!
len(raw_eeg.ch_names)

#check again!
raw_eeg.info
raw_eeg.plot(events=events, event_id=event_id)

In [ ]:
###Crop and filter the data
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

raw_eeg_cropped.load_data()
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40)

##for comparison:
raw_eeg_cropped.plot(events=events, event_id=event_id)
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id)


import matplotlib.pyplot as plt
fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

In [ ]:

##Save the data
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif',
                              overwrite=True)


#BIDS:brain image data structure

In [ ]:
#https://mne-tools/mne-bids


import matplotlib
import pathlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')


In [ ]:

#raw data
sample_data_dir = mne.datasets.sample.data_path()
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / '....fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

raw.info()

In [ ]:
#we can addd sth beased on the mne.tools website instructions:

subject_info={
    'birthday' : (1998 , 10 , 1),
    'sex' : 2       #2 is for female!
    'hand' : 3      #3 for abmivent hand(both right and left handed)
}

raw.info['subject_info']=subject_info

#let's check
raw.info                   #it should be added

In [ ]:
##Write the raw data to BIDS!!

raw.info['line_freq'] = 60          ##based on the lab which you work in!

out_path = pathlib.Path('out_data/sample_BIDS')
bids_path = mne_bids.BIDSPath(subject='01',            #these are the things in the BIDS files!!
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True , annonymize=None)
 #annonomyzie is for shifting the time the data was recorded and to delete some of the meta data whcih you don't need

In [ ]:

##having a summary!!
mne_bids.print_dir_tree(out_path)
print(mne_bids.make_report(out_path))

In [ ]:
##Reading BIDS data
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

raw.plot()


#mne.bids will add the event it finds as an annotations to the raw file!!

In [ ]:
####Events are stored as annotations – but we convert between the two.

raw.annotations[0]
events, event_id = mne.events_from_annotations(raw)

mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])


#finding the paths:

bids_path.meg_calibration_fpath
bids_path.meg_crosstalk_fpath

In [ ]:
#Creating epochs and generating evoked responses (ERP/ERF)

bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()


raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)


tmin = -0.3
tmax = 0.5
baseline = (None, 0)          #None means from the beginning of the epoch    0 means till the event

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,       #if you say the baseline to be None here it won't do the baseline correction nut we wants it here!
                    preload=True)            #to load the data to the memory
epochs

epochs.plot()


In [ ]:

# we can select  epochs based on experimental conditions
epochs['Auditory/Right']
epochs['Auditory']
epochs['Left']
epochs['Visual'].plot_image()

##let's selcet the right epochs from the eeg only:
epochs['Right'].copy().pick_types(meg=False , eog=False ,eeg=True).plot_image()

#or
epochs['Right'].plot_image(picks=['EEG'])

#Saving epochs
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif',overwrite=True)          #for epochs use """_epo""" !

In [ ]:
##Creating evoked data=-----average of epocks !

evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

#VISUALIZATION!!
evoked_auditory.plot(spatial_colors=True)
evoked_auditory.plot_topomap(ch_type='mag' , times=[0,0.1 , 0.2 , 0.3 , 0.4 ,0.5])
evoked_auditory.plot_joint(picks='mag')
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag')

##Saving evoked data
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                  evoked=[evoked_auditory, evoked_visual])


##Reading evoked data
#ALL:
evokeds = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif')

evokeds

#JUST ONE :
evokeds[0]
evoked = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                          condition='0.50 * Visual/Left + 0.50 * Visual/Right')
evoked

#CLEANING----DELETING ARTIFACTS

In [ ]:
epochs.drop_bad

epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

epochs.apply_baseline((None, 0))
epochs.plot()


#Reject artifacts based on channel signal amplitude:

reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV


epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)
epochs.plot_drop_log()

epochs['Visual'].plot_image()
epochs.plot_sensors(ch_type='eeg')
epochs['Visual'].plot_image(picks='EEG 060')

In [ ]:
#1)
####SSP:signal space projection
#Let's see if we can retain most the epochs, but still get rid of the EOG artifact. And while we're at it, the ECG artifacts too 😉
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()

raw.filter(l_freq=0.1, h_freq=40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0,             #ecg artifacts
                                                           average=True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1,n_eeg=1,             #eog artifacts
                                                           average=True)

ecg_proj
eog_projs

projs = eog_projs + ecg_projs
projs

epochs.add_proj(projs)
epochs.plot()


epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks='EEG 060')

#2)
####ICA
##for removing eog (eye blinks)  & ecg (hearbeats)
#First, start with the raw data again and apply a 1.0 Hz high-pass filter, which is advantegeous for ICA performance.

bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40)  # High-pass with 1. Hz cut-off is recommended for ICA

epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

epochs.plot()--------#we cn delete some data or ba channels here

epochs_selection    --------------#do it again &  will see some epochs are gone!!

#Only keep the subset of events that corresponds to the retained epochs.
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

events.shape()


#Create epochs for ICA. All parameters should match exactly the ones of the epochs we intend to clean.

tmin = epochs.tmin
tmax = epochs.tmax
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True)



#Finally, fit ICA!

epochs_ica.info


n_components = 0.8                     # Should normally be higher, like 0.999!!
method = 'picard'                      #3 methods 3 algorithms : fast ICA , info max ICA ,picard ICA
max_iter = 100                         # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

ica = mne.preprocessing.ICA(n_components=n_components,
                            max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)


ica.fit(epochs_ica)

ica.plot_components(inst=epochs)


##Detect ECG and EOG patterns
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),           #starts at the first of the epoch and ends 0.2 seconds before the end of the heart beat
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(ecg_epochs, method='ctps')


eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(eog_epochs , method='ctps')


components_to_exclude = ecg_inds + eog_inds
ica.exclude = components_to_exclude

ica.exclude

#Plot automated artifact detection scores------when ICA is done it gives scores for the bad channels it finds!!!
ica.plot_scores(ecg_scores)
ica.plot_scores(eog_scores)

#Plot ICA sources
ica.plot_sources(ecg_evoked)


#Plot overlay of original and cleaned data
ica.plot_overlay(ecg_evoked)




###NOW CLEAN THE DATA(DELETE THOSE BAD CHANNELS IVA FOUND!)

epoches_cleaned=ica.apply(epochs.copy())

#for comparison:
epoches_cleaned.plot(title='after ICA')
epoches.plot(title='before ICA')


epoches_cleaned.save('cleaned epoches after ICA')

##WELL DONE TILL HERE !!

In [ ]:
###NOW :
#Let's Load EEGLAB epochs and setting the proper montage (electrode locations) for EEG data

epochs_fname = pathlib.Path('eeg_montage_data') / 'sample_eeglab_epochs.set'
epochs = mne.read_epochs_eeglab(epochs_fname)

epochs

epochs.plot_sensors()

#Load vendor-supplied montage:where were the sensors on the persons head!!

montage_fname = pathlib.Path('eeg_montage_data') / 'achtiCHamp_64_channels_and_fiducials_Theta_Phi.txt'
dig_montage = mne.channels.read_custom_montage(montage_fname)
dig_montage.plot(sphere='auto')

#Apply this montage to the loaded epochs
epochs.set_montage(dig_montage)

epochs.plot_sensors()


#ANALYSIS

In [ ]:
##NOW ANALYSIS!!!!!

#Multivariate Statistics (Decoding / MVPA) on MEG/EEG Data
%matplotlib qt
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')


epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs.apply_baseline((None, 0))

epochs


##Select epochs of interest.Here, we intend to analyze the auditory epochs.

epochs_auditory = epochs['Auditory']

epochs_auditory

#Calculate empirical evoked difference

evoked_diff = mne.combine_evoked(
    [epochs_auditory['Auditory/Left'].average(),
     epochs_auditory['Auditory/Right'].average()],
     weights=[1, -1]  # Subtraction
)

evoked_diff.plot(gfp=True)

mne.viz.plot_compare_evokeds(
    [epochs_auditory['Auditory/Left'].average(),
     epochs_auditory['Auditory/Right'].average(),
     evoked_diff]
)



SOME ML!!!!

In [ ]:
###Okay, but… we want more than that! Let's do some machine learning!!!!

#Equalize the number of epochs
#To keep chance level at 50% accuracy, we first equalize the number of epochs in each condition.

epochs_auditory.equalize_event_counts(epochs_auditory.event_id)

epochs_auditory


#Create input X and response y.
#A classifier takes as input a matrix X and returns a vector y (consisting of 0 and 1).
#Here X will be the data at one time point on all gradiometers (hence the term multivariate).
#We want to train our model to discriminate between the Auditory/Left and the Auditory/Right trials.
#We work with all sensors jointly and try to find a discriminative pattern between the two conditions to predict the experimental condition of individual trials.

import numpy as np

# Create an vector with length = no. of trials.
y = np.empty(len(epochs_auditory.events), dtype=int)

# Which trials are LEFT, which are RIGHT?
idx_left = epochs_auditory.events[:, 2] == epochs_auditory.event_id['Auditory/Left']
idx_right = epochs_auditory.events[:, 2] == epochs_auditory.event_id['Auditory/Right']

#we'll have a list of TRUE & FALSEs
idx_left
idx_right


# Encode: LEFT = 0, RIGHT = 1.
y[idx_left] = 0
y[idx_right] = 1

print(y)
print(f'\nSize of y: {y.size}')


#Now, let's create the input matrix, X.

#We wish to focus only on the gradiometer channels here, so we use pick_types(meg='grad').
#For magnetometer channels, we would need to pass meg='mag'; and for EEG channels: meg=False, eeg=True.
#We create a copy of the epochs because pick_types() operates in-place, but we would like to keep the original epochs object untouched.


epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')


data = epochs_auditory_grad.get_data()         # The array has the shape: (n_trials, n_channels, n_timepoints)
print(data.shape)

#We need to reshape the array such that for each trial, we have a vector [channel_1_time_1, channel_1_time_2, ..., channel_m_time_n], i.e.,
#we aim to reshape X to the dimension (n_trials, n_channels * n_timepoints).

n_trials = data.shape[0]

X = data.reshape(n_trials, -1)
print(X)
print(X.shape)


In [ ]:

#####NOW:
#Create a classifier!
#We will use plain scikit-learn machinery for the first round of classifications.
#This is to demonstrate that you can simply feed pre-processed data from MNE into scikit-learn.

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


# The classifier pipeline: it is extremely important to scale the data
# before running the actual classifier (logistic regression in our case).
clf = make_pipeline(StandardScaler(),LogisticRegression())

# Run cross-validation.
# CV without shuffling – "block cross-validation" – is what we want here

n_splits = 5
scoring = 'roc_auc'
cv = StratifiedKFold(n_splits=n_splits)
scores = cross_val_score(clf, X=X, y=y, cv=cv, scoring=scoring)

# Mean and standard deviation of ROC AUC across cross-validation runs.
roc_auc_mean = round(np.mean(scores), 3)
roc_auc_std = round(np.std(scores), 3)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f} (SD = {roc_auc_std:.3f})')


#Visualize the cross-validation results.
fig, ax = plt.subplots()
ax.boxplot(scores,
           showmeans=True, # Green triangle marks the mean.
           whis=(0, 100),  # Whiskers span the entire range of the data.
           labels=['Left vs Right'])
ax.set_ylabel('Score')
ax.set_title('Cross-Validation Scores')


#####We can do this more simply using the mne.decoding module! Let's go. 🚀
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore


# First, create X and y.
epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')
X = epochs_auditory_grad.get_data()
y = epochs_auditory_grad.events[:, 2]

# Classifier pipeline.
clf = make_pipeline(
                                       # An MNE scaler that correctly handles different channel types –
   Scaler(epochs_grad.info),
                                        # Remember this annoying and error-prone NumPy array reshaping we had to dO earlier? Not anymore, thanks to the MNE vectorizer!
    Vectorizer(),
                                         # And, finally, the actual classifier.
    LogisticRegression())

# Run cross-validation.
# Note that we're using MNE's cross_val_multiscore() here, not scikit-learn's
# cross_val_score() as above. We simply pass the number of desired CV splits,
# and MNE will automatically do the rest for us.

n_splits = 5
cv=KFOLD(random_state=32 , n_splits=5)
scoring = 'roc_auc'
scores = cross_val_multiscore(clf, X, y, cv=cv, scoring='roc_auc')

# Mean and standard deviation of ROC AUC across cross-validation runs.
roc_auc_mean = round(np.mean(scores), 3)
roc_auc_std = round(np.std(scores), 3)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f}' , f'SD = {roc_auc_std:.3f}')

In [ ]:
##Decoding over time: Comparisons at every single time point.
"""
#In the previous examples, we have trained a classifier to discriminate between experimental conditions by using the spatio-temporal patterns of entire trials.
#Consequently, the classifier was (hopefully!) able to predict which activation patterns belonged to which condition.
#However, an interesting neuroscientific is: Exactly when do the brain signals for two conditions differ?
#We can try to answer this question by fitting a classifier at every single time point.
#If the classifier can successfully discriminate between the two conditions,
#we can conclude that the spatial activation patterns measured by the MEG or EEG sensors differed at this very time point.
"""

from sklearn.preprocessing import StandardScaler
from mne.decoding import SlidingEstimator

# First, create X and y.
epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')
X = epochs_auditory_grad.get_data()
y = epochs_auditory_grad.events[:, 2]

# Classifier pipeline. No need for vectorization as in the previous example.
clf = make_pipeline(StandardScaler(),LogisticRegression())

# The "sliding estimator" will train the classifier at each time point.
scoring = 'roc_auc'
time_decoder = SlidingEstimator(clf, scoring=scoring, n_jobs=1, verbose=True)

# Run cross-validation.
n_splits = 5
scores = cross_val_multiscore(time_decoder, X, y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits, for each time point.
mean_scores = np.mean(scores, axis=0)

# Mean score across all time points.
mean_across_all_times = round(np.mean(scores), 3)
print(f'\n=> Mean CV score across all time points: {mean_across_all_times:.3f}')


##Plot the classification results!
fig, ax = plt.subplots()

ax.axhline(0.5, color='k', linestyle='--', label='chance')    # AUC = 0.5
ax.axvline(0, color='k', linestyle='-')                       # Mark time point zero.
ax.set_xlabel('Time (s)')
ax.set_ylabel('Mean ROC AUC')
ax.legend()
ax.set_title('Left vs Right')
fig.suptitle('Sensor Space Decoding')
ax.plot(epochs.times, mean_scores, label='score')


In [ ]:
##Frequency and time-frequency sensors analysis##

#The objective is to show you how to explore the spectral content of your data (frequency and time-frequency).
------------------------------------------------------------
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')


#Set parameters
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')

epochs.apply_proj()
epochs_auditory = epochs['Auditory']


#Frequency analysis
#Let's first check out all channel types by averaging across epochs.

epochs_auditory.plot_psd(fmin=2., fmax=40., average=True, bandwidth=2)


#Select a frequency range in the plot to inspect topographies
#The "bandwidth" parameter controls the spectral resolution of the multitaper.
#You can increase the resolution by chosing a narrower bandwidth at the cost of longer computation time.
#Now let's take a look at the spatial distributions of the PSD.

epochs_auditory.plot_psd_topomap(ch_type='eeg', normalize=False)
epochs_auditory.plot_psd_topomap(ch_type='grad', normalize=True)       ------#different in the visualization a bit with the Normalization!
epochs_auditory.plot_psd_topomap(ch_type='mag', normalize=False ,      ------#you can do it with defining the bands!
                                 bands=[(0,10 , 'custom1'),
                                        (7.5 , 15 , 'custom2')])


##Time-frequency analysis: power and inter-trial coherence!!
#We now compute time-frequency representations (TFRs) from our Epochs.
#We'll look at power and inter-trial coherence (ITC).
#To this we'll use the function mne.time_frequency.tfr_morlet but you can also use mne.time_frequency.tfr_multitaper or mne.time_frequency.tfr_stockwell.

import numpy as np

# define frequencies of interest (log-spaced)

freqs = np.logspace(*np.log10([2, 30]), num=20)
freqs

n_cycles = freqs / 2.            # different number of cycle per frequency

power, itc = mne.time_frequency.tfr_morlet(epochs_auditory, freqs=freqs, n_cycles=n_cycles,
                                           use_fft=True,return_itc=True, decim=3, n_jobs=1)

power.crop(-0.1, 0.7)            # crop to remove edge artifacts
itc.crop(-0.1, 0.7)              # crop to remove edge artifacts



###Inspect power
#In the topo you can click on an image to visualize the data for one sensor.
#You can also select a portion in the time-frequency plane to obtain a topomap for a certain time-frequency region.

baseline_mode = 'logratio'
baseline = (None, 0)

power.copy().pick_types(eeg=True, meg=False).plot_topo()

   #Plot power of an individual channel
power.plot(picks='EEG 050', baseline=baseline, mode=baseline_mode)
  #Plot topomaps for specified frequency ranges
import matplotlib.pyplot as plt

fig, axis = plt.subplots(1, 3, figsize=(7, 4))
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=4, fmax=7,
                   baseline=baseline, mode=baseline_mode, axes=axis[0],
                   title='Theta', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=8, fmax=12,
                   baseline=baseline, mode=baseline_mode, axes=axis[1],
                   title='Alpha', show=False, contours=1)
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=15, fmax=30,
                   baseline=baseline, mode=baseline_mode, axes=axis[2],
                   title='Beta', show=False, contours=1)
mne.viz.tight_layout()
plt.show()

####Joint Plot
#You can also create a joint plot showing both the aggregated TFR across channels and topomaps at specific times and frequencies
#to obtain a quick overview regarding oscillatory effects across time and space.

power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.05, 2.), (0.1, 11.)])
plt.show()


####Inspect ITC
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=0.5, cmap='Reds')

##Baseline correction can be applied to power or done in plots.
##To illustrate the baseline correction in plots:
 power.apply_baseline(baseline=(-0.5, 0), mode='logratio')


"""SOURCE ESTIMATION----EEG TO MRI!!"""

In [ ]:
"""SOURCE ESTIMATION----EEG TO MRI!!"""

#NOW:
####Source estimation

%matplotlib qt
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne_bids
import mne

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

#Start with some fresh epochs
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)

tmin = -0.200
tmax = 0.500
baseline = (None, 0)

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True,
                    proj=False)

epochs.save(pathlib.Path('out_data') / 'epochs_for_source_epo.fif')
epochs.info


###The most important part###
##View the BEM
subjects_dir = pathlib.Path(mne.datasets.sample.data_path()) / 'subjects'

mne.viz.plot_bem(subject='sample', subjects_dir=subjects_dir, orientation='coronal')     #like an MRI gives you the location!!


##co-registration:have the positions of sensors of EEG on the MRI pictures:

epochs_fname = pathlib.Path('out_data') / 'epochs_for_source_epo.fif'

mne.gui.coregistration(subject='sample', subjects_dir=subjects_dir,inst=epochs_fname)


trans_fname = pathlib.Path('out_data') / 'sample_new.trans.fif'  --#the path it saves in our pc
info = mne.io.read_info(epochs_fname)
mne.viz.set_3d_backend('pyvista')

fig = mne.viz.plot_alignment(info=info, trans=trans_fname, subject='sample', dig=True,
                             subjects_dir=subjects_dir, verbose=True)

##Compute the source space
subject = 'sample'
src = mne.setup_source_space(subject=subject,
                             spacing='oct4',  # Use oct6 during an actual analysis!
                             subjects_dir=subjects_dir,
                             add_dist=False)  # Remove this one during an actual analysis!

src


mne.viz.plot_alignment(info=info, trans=trans_fname, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

##Compute the forward solution!
conductivity = (0.3,)               # for single layer – used in MEG
conductivity = (0.3, 0.006, 0.3)    # for three layers – used in EEG

model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
model

bem_sol = mne.make_bem_solution(model)
bem_sol

bem_fname = pathlib.Path('out_data') / 'sample_bem.fif'
mne.bem.write_bem_solution(bem_fname, bem_sol)
fwd = mne.make_forward_solution(raw_fname,
                                trans=trans_fname,
                                src=src,
                                bem=bem_sol,
                                meg=True, # include MEG channels
                                eeg=False, # exclude EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel
fwd

fwd_fname = pathlib.Path('out_data') / 'sample_fwd.fif'
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)


##Compute noise covariance
noise_cov = mne.compute_covariance(epochs, tmax=0.,
                                   method=['shrunk', 'empirical'],
                                   rank='info')

mne.viz.plot_cov(noise_cov, info=info)

epochs_average().plot_white(noise_cov)

##Create the inverse operator
from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

fwd_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-meg-fwd.fif')
fwd = mne.read_forward_solution(fwd_fname)
fwd = mne.convert_forward_solution(fwd, surf_ori=True)

# Restrict forward solution as necessary for MEG
fwd = mne.pick_types_forward(fwd, meg=True, eeg=False)

# make an M/EEG, MEG-only, and EEG-only inverse operator
info = contrast.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

#Apply the inverse operator -> Calculate the source estimate
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(epochs['Auditory/Left'], inverse_operator, lambda2,
                    method=method, pick_ori=None)

brain = stc.plot(surface='inflated',                    #all the brain:inflated , the cortex:pial  ,white matter:white
                 hemi='both',
                 subjects_dir=subjects_dir,
                 time_viewer=True)